In [6]:
from PIL import Image, ImageDraw
import pytesseract
bounding_boxes=[]
def process_image(input_path, output_path):
    # Open the image file
    img = Image.open(input_path)

    #img_gray = img.convert('L')

    # Get the text and bounding box information from the grayscale image using Tesseract OCR
    result = pytesseract.image_to_boxes(img, config='--psm 8')


    # Create an image with the same size as the input image
    output_img = img.copy()
    draw = ImageDraw.Draw(output_img)

    # Get image height for coordinate transformation
    img_height = img.size[1]

    # Initialize variables for tracking bounding box position
    prev_rightmost_x = 0

    # Iterate through each character and draw a bounding box
    for char_info in result.splitlines():
        char_data = char_info.split()
        char = char_data[0]

        if char.isalpha():
            # Get the bounding box for the character
            bbox = [int(coord) for coord in char_data[1:5]]

            # Transform coordinates to PIL coordinate system
            bbox[1], bbox[3] = img_height - bbox[3], img_height - bbox[1]
            print(f"letter: {char}")
            print(f"prev: {prev_rightmost_x}")
            print(f"left coord: {bbox[0]}")
            print(f"right coord: {bbox[2]}")
            # Adjust x-coordinates to prevent overlap

            if(prev_rightmost_x<bbox[2]):
                bbox[0] = max(bbox[0], prev_rightmost_x)
            bounding_boxes.append(bbox)
            # Draw a rectangle around the character in the output image with red color
            

            # Update the rightmost x-coordinate for the next bounding box
            prev_rightmost_x = bbox[2]

    # Save the output image
    adjust_bbox_order()
    for bbox in bounding_boxes:
        print(bbox)
    draw_on_image(input_path, output_path)

    
    # Return the array of letters
    letters = [char.upper() for char in result if char.isalpha()]
    return letters

def draw_on_image(input_path, output_path):
    img = Image.open(input_path)

     # Create an image with the same size as the input image
    output_img = img.copy()
    draw = ImageDraw.Draw(output_img)
    for bbox in bounding_boxes:
        draw.rectangle(bbox, outline='red')

    output_img.save(output_path)

def adjust_bbox_order():
    for i in range(len(bounding_boxes) - 1):
        current_bbox = bounding_boxes[i]
        next_bbox = bounding_boxes[i + 1]

        # Extracting coordinates
        current_right = current_bbox[2]
        next_left = next_bbox[0]

        # Checking the condition
        if current_right > next_left:
            # Adjusting bbox[2] to the minimum value
            bounding_boxes[i][2] = min(current_right, next_left)

    #return bounding_boxes


if __name__ == "__main__":
    input_image_path = r"supervised/century_schoolbook_images/century_schoolbook-018.jpg"
    output_image_path = r"boundingboxes\cent-018.jpg"

    result = process_image(input_image_path, output_image_path)
    print("Array of letters:", result)


Array of letters: []


In [ ]:
input_image_path = r"supervised\century_schoolbook_images\century_schoolbook-011.jpg"
output_image_path = r"boundingboxes\cent-011.jpg"


In [1]:
from PIL import Image, ImageDraw
import pytesseract
import os
import csv

bounding_boxes = []


def process_image(input_path, output_path):
    # Open the image file
    img = Image.open(input_path)

    # Get the text and bounding box information from the image using Tesseract OCR
    result = pytesseract.image_to_boxes(img, config='--psm 11 --oem 1')

    # Create an image with the same size as the input image
    output_img = img.copy()
    draw = ImageDraw.Draw(output_img)

    # Get image height for coordinate transformation
    img_height = img.size[1]

    # Initialize variables for tracking bounding box position
    prev_rightmost_x = 0

    # Iterate through each character and draw a bounding box
    for char_info in result.splitlines():
        char_data = char_info.split()
        char = char_data[0]

        if char.isalpha():
            # Get the bounding box for the character
            bbox = [int(coord) for coord in char_data[1:5]]

            # Transform coordinates to PIL coordinate system
            bbox[1], bbox[3] = img_height - bbox[3], img_height - bbox[1]

            # Adjust x-coordinates to prevent overlap
            if prev_rightmost_x < bbox[2]:
                bbox[0] = max(bbox[0], prev_rightmost_x)
            bounding_boxes.append(bbox)

            # Update the rightmost x-coordinate for the next bounding box
            prev_rightmost_x = bbox[2]

    # Save the output image
    adjust_bbox_order()
    draw_on_image(output_img, output_path)

    # Return the array of letters
    letters = [char.upper() for char in result if char.isalpha()]
    return letters


def draw_on_image(image, output_path):
    draw = ImageDraw.Draw(image)
    for bbox in bounding_boxes:
        draw.rectangle(bbox, outline='red')

    image.save(output_path)


def adjust_bbox_order():
    for i in range(len(bounding_boxes) - 1):
        current_bbox = bounding_boxes[i]
        next_bbox = bounding_boxes[i + 1]

        # Extracting coordinates
        current_right = current_bbox[2]
        next_left = next_bbox[0]

        # Checking the condition
        if current_right > next_left:
            # Adjusting bbox[2] to the minimum value
            bounding_boxes[i][2] = min(current_right, next_left)


def save_to_csv(csv_path, image_name):
    with open(csv_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        

        for bbox in bounding_boxes:
            csv_writer.writerow([image_name] + bbox)


def process_directory(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, file)
                output_path = os.path.join(output_dir, file)
                csv_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}.csv")

                # Process the image
                process_image(input_path, output_path)

                # Save bounding box values to CSV
                save_to_csv(csv_path, file)

                # Clear bounding_boxes for the next image
                bounding_boxes.clear()


if __name__ == "__main__":
    input_directory = "supervised/century_schoolbook_images/"
    output_directory = "supervised_with_bbox/century_schoolbook_images"

    # Create output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Process all images in the input directory
    process_directory(input_directory, output_directory)
    print("Processing complete.")


Processing complete.


In [3]:
import os
import pandas as pd

def join_csv_files_and_output(folder_path, output_file):
    # Get a list of all CSV files in the folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    # Initialize an empty DataFrame to store the joined data
    joined_df = pd.DataFrame()

    # List to store names of empty CSV files
    empty_csv_files = []

    # Loop through each CSV file
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)

        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Check if the DataFrame is empty
            if df.empty:
                empty_csv_files.append(csv_file)
            else:
                # Join the non-empty DataFrame to the main DataFrame
                joined_df = joined_df.append(df, ignore_index=True)
        except pd.errors.EmptyDataError:
            # Handle EmptyDataError by appending the file to empty_csv_files
            empty_csv_files.append(csv_file)

    # Print the names of empty CSV files
    if empty_csv_files:
        print("Empty CSV files:")
        for empty_file in empty_csv_files:
            print(empty_file)
    else:
        print("No empty CSV files.")

    # Output the joined DataFrame to a new CSV file
    if not joined_df.empty:
        joined_df.to_csv(output_file, index=False)
        print(f"Joined data saved to {output_file}")
    else:
        print("No non-empty CSV files to join.")

# Example usage:
folder_path = 'supervised_with_bbox/century_schoolbook_images/'
output_file = 'supervised_with_bbox/century_schoolbook.csv'
join_csv_files_and_output(folder_path, output_file)


C:\Users\seher\AppData\Local\Temp\ipykernel_38012\2808192502.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = joined_df.append(df, ignore_index=True)
C:\Users\seher\AppData\Local\Temp\ipykernel_38012\2808192502.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = joined_df.append(df, ignore_index=True)
C:\Users\seher\AppData\Local\Temp\ipykernel_38012\2808192502.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = joined_df.append(df, ignore_index=True)
C:\Users\seher\AppData\Local\Temp\ipykernel_38012\2808192502.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = joined_df.append(df,

Empty CSV files:
century_schoolbook-018.csv
century_schoolbook-019.csv
century_schoolbook-022.csv
century_schoolbook-025.csv
century_schoolbook-026.csv
century_schoolbook-051.csv
century_schoolbook-054.csv
century_schoolbook-066.csv
century_schoolbook-077.csv
century_schoolbook-084.csv
century_schoolbook-087.csv
century_schoolbook-089.csv
century_schoolbook-093.csv
century_schoolbook-097.csv
century_schoolbook-1000.csv
century_schoolbook-1015.csv
century_schoolbook-1017.csv
century_schoolbook-103.csv
century_schoolbook-1044.csv
century_schoolbook-1049.csv
century_schoolbook-1066.csv
century_schoolbook-1077.csv
century_schoolbook-1088.csv
century_schoolbook-1090.csv
century_schoolbook-1095.csv
century_schoolbook-1097.csv
century_schoolbook-111.csv
century_schoolbook-117.csv
century_schoolbook-120.csv
century_schoolbook-121.csv
century_schoolbook-127.csv
century_schoolbook-131.csv
century_schoolbook-135.csv
century_schoolbook-152.csv
century_schoolbook-166.csv
century_schoolbook-218.csv
